In [2]:
from src.handle_data import split_data

In [3]:
data_path = "data/boston_housing_data.csv"
# use same random_state across all parts to ensure reproducibility
random_state = 20230325

# Splits the data into X and y train and test portions
X_train, X_test, y_train, y_test = split_data(data_path, proportion = 0.75, target = "MEDV", random_state=random_state)